In [29]:
import numpy as np
import pandas as pd
from astropy.coordinates import angular_separation

BCGs = pd.read_excel('full_BCG_list.xlsx')
BCGs = BCGs.to_numpy(dtype=str)
BCGsClusterNames = BCGs.T[0]
BCGsClusterNames = np.char.strip(BCGsClusterNames)

for i, val in enumerate(BCGs.T[5]): #some of the entries have things like "x update: y", these loops take only the y
    BCGs.T[5][i] = val.split()[-1]
BCG_RAs = BCGs.T[5]

for i, val in enumerate(BCGs.T[6]):
    BCGs.T[6][i] = val.split()[-1]
BCG_DECs = BCGs.T[6]

for i, val in enumerate(BCGs.T[7]):
    BCGs.T[7][i] = val.split()[-1]
BCG_Zs = BCGs.T[7]

#get global props data
globalClusters = np.loadtxt(r"../tools/data/globalTestFormat.csv",delimiter=',',dtype=str)
globalNames = globalClusters.T[0][3:]
globalNames = np.char.strip(globalNames)
globalRAs = dict(zip(globalNames,globalClusters.T[8][3:]))
globalDECs = dict(zip(globalNames,globalClusters.T[9][3:]))
globalZs = dict(zip(globalNames,globalClusters.T[10][3:]))

#get profile K0
profiles = np.loadtxt(r"../tools/data/ACCEPT2_K0_and_z.csv",delimiter=',',dtype=str)
profilesNames = profiles.T[1][1:]
profilesNames = np.char.strip(profilesNames)
profileK0s = dict(zip(profilesNames,profiles.T[2][1:]))

In [30]:
#initialize empty table
table = np.zeros((len(BCGsClusterNames),5),dtype=object)

#set first column to all accept2.0 names
table[:,0]=BCGsClusterNames

In [34]:
for i, name in enumerate(BCGsClusterNames): #goes through each cluster and compares the coords
    coords = np.radians((float(globalRAs[name]),float(globalDECs[name]),float(BCG_RAs[i]),float(BCG_DECs[i]))) #converts coords to radians
    separation = angular_separation(*coords) #uses astropy to calculate
    table[i,1] = np.degrees(separation)*3600 #converts separation to arcseconds
    table[i,2] = np.abs(float(BCG_Zs[i])-float(globalZs[name]))/float(globalZs[name])
    table[i,3] = np.abs(float(BCG_Zs[i])-float(globalZs[name]))/(1+float(globalZs[name]))
    try: table[i,4] = float(profileK0s[name]) #sees if there is a K0
    except: table[i,4] = np.nan

In [40]:
df = pd.DataFrame(table,columns=['Cluster Name', 'Angular Separation (arcseconds)', 'Z Separation', 'Z Separation over +1', "K0"])
df = df.sort_values('Angular Separation (arcseconds)',ascending=False)
df.to_csv("bcg_coords_compare.csv",index=None)